In [1]:
import os
import cv2
import matplotlib.pyplot as plt

# Function to detect and crop faces from a frame
def crop_faces(frame):
    # Load the Haar cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Crop faces from the frame
    cropped_faces = []
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        cropped_faces.append(face)
    
    return cropped_faces

# Function to preprocess a video
def preprocess_video(input_path, output_path):
    # Create output directory if it doesn't exist
    #if not os.path.exists(output_path):
    #    os.makedirs(output_path)
    
    # Open the video file
    cap = cv2.VideoCapture(input_path)
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Determine frame skip interval to get approximately 100 frames
    #skip_interval = max(1, total_frames // 100)
    
    # Create VideoWriter object to save the preprocessed video in MP4 format
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # XVID codec
    out = cv2.VideoWriter(output_path, fourcc, fps, (128, 128))
    #out = cv2.VideoWriter(output_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (224,224))
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Skip frames based on skip_interval
        #if frame_number % skip_interval != 0:
        #    frame_number += 1
        #    continue
        
        # Crop faces from the frame
        cropped_faces = crop_faces(frame)
        
        # Resize and save each cropped face
        for face in cropped_faces:
            if face.shape[0] > 0 and face.shape[1] > 0:
                resized_face = cv2.resize(face, (128, 128))
                #normalized_face = resized_face/255.0
                #plt.imshow(resized_face)
                out.write(resized_face)
        
        frame_count += 1
        
        # Limit to 10 frames per video
        if frame_count >= 20:
            break
    
    # Release video capture and writer objects
    cap.release()
    out.release()

# Preprocess videos in 'Celeb-real' folder
input_folder_real = 'Celeb-real'
output_folder_real = 'Celeb-real-preprocessed'
for video_file in os.listdir(input_folder_real):
    input_path = os.path.join(input_folder_real, video_file)
    output_path = os.path.join(output_folder_real, video_file)
    preprocess_video(input_path, output_path)

# Preprocess videos in 'Celeb-synthesis' folder
input_folder_synthesis = 'Celeb-synthesis'
output_folder_synthesis = 'Celeb-synthesis-preprocessed'
for video_file in os.listdir(input_folder_synthesis):
    input_path = os.path.join(input_folder_synthesis, video_file)
    output_path = os.path.join(output_folder_synthesis, video_file)
    preprocess_video(input_path, output_path)


In [2]:
import os
import random
import shutil
import cv2

# Function to extract frames from a video file
def extract_frames(video_path, target_dir):
    cap = cv2.VideoCapture(video_path)
    success, image = cap.read()
    count = 0

    while success:
        frame_path = os.path.join(target_dir, f"frame_{count}.jpg")
        cv2.imwrite(frame_path, image)
        success, image = cap.read()
        count += 1

    cap.release()

# Function to split frames into train and test sets
def split_frames(source_dir, target_train_dir, target_test_dir, split_ratio=0.8):
    video_files = os.listdir(source_dir)
    for video_file in video_files:
        video_path = os.path.join(source_dir, video_file)
        frames_dir = os.path.splitext(video_file)[0]  # Use video filename as frames directory
        frames_dir_path = os.path.join(source_dir, frames_dir)
        os.makedirs(frames_dir_path, exist_ok=True)

        # Extract frames from video
        extract_frames(video_path, frames_dir_path)

        # Split frames into train and test sets
        frames = os.listdir(frames_dir_path)
        random.shuffle(frames)  # Shuffle frames randomly
        num_frames = len(frames)
        train_frames = frames[:int(num_frames * split_ratio)]
        test_frames = frames[int(num_frames * split_ratio):]

        # Create subdirectories in target directories if they don't exist
        os.makedirs(os.path.join(target_train_dir, frames_dir), exist_ok=True)
        os.makedirs(os.path.join(target_test_dir, frames_dir), exist_ok=True)

        # Move frames to train and test directories
        for frame in train_frames:
            shutil.move(os.path.join(frames_dir_path, frame), os.path.join(target_train_dir, frames_dir, frame))
        for frame in test_frames:
            shutil.move(os.path.join(frames_dir_path, frame), os.path.join(target_test_dir, frames_dir, frame))

# Example usage
real_preprocessed_dir = 'Celeb-real-preprocessed'
deepfake_preprocessed_dir = 'Celeb-synthesis-preprocessed'

# Define target directories
target_dir = 'dataset'
train_dir = os.path.join(target_dir, 'train')
test_dir = os.path.join(target_dir, 'test')
split_ratio = 0.8
split_frames(real_preprocessed_dir, os.path.join(train_dir, 'real'), os.path.join(test_dir, 'real'), split_ratio)
split_frames(deepfake_preprocessed_dir, os.path.join(train_dir, 'deepfake'), os.path.join(test_dir, 'deepfake'), split_ratio)


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define directories
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Define image size and batch size
image_size = (128, 128)  # Adjusted frame size
batch_size = 32

# Use data augmentation for training images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# No data augmentation for validation/test images
test_datagen = ImageDataGenerator(rescale=1./255)

# Generate batches of augmented image data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Assuming binary classification (real vs deepfake)
)

# Generate batches of validation/test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Assuming binary classification (real vs deepfake)
    shuffle=False  # No shuffling for test data
)

# Load pre-trained EfficientNetB0 model without top layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add top layers for classification
x = GlobalAveragePooling2D()(base_model.output)
output = Dense(1, activation='sigmoid')(x)

# Create model
model = Model(base_model.input, output)

# Compile the model with the correct learning rate argument
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

# Evaluate the trained model
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
model.save("my_model.h5")


Found 14945 images belonging to 2 classes.
Found 3908 images belonging to 2 classes.
Epoch 1/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 676s 1s/step - accuracy: 0.8030 - loss: 0.4419 - val_accuracy: 0.8279 - val_loss: 5.2276
Epoch 2/10
  1/467 ━━━━━━━━━━━━━━━━━━━━ 9:37 1s/step - accuracy: 0.9375 - loss: 0.1921

C:\Users\rakul\anaconda3\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


467/467 ━━━━━━━━━━━━━━━━━━━━ 2s 915us/step - accuracy: 0.9375 - loss: 0.0962 - val_accuracy: 0.0000e+00 - val_loss: 14.9310
Epoch 3/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 602s 1s/step - accuracy: 0.9337 - loss: 0.1760 - val_accuracy: 0.9613 - val_loss: 0.0994
Epoch 4/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 1s 239us/step - accuracy: 0.9375 - loss: 0.0489 - val_accuracy: 1.0000 - val_loss: 0.1016
Epoch 5/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 592s 1s/step - accuracy: 0.9546 - loss: 0.1219 - val_accuracy: 0.9818 - val_loss: 0.0533
Epoch 6/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 1s 243us/step - accuracy: 0.9688 - loss: 0.0201 - val_accuracy: 0.7500 - val_loss: 0.1973
Epoch 7/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 590s 1s/step - accuracy: 0.9635 - loss: 0.0915 - val_accuracy: 0.9851 - val_loss: 0.0379
Epoch 8/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 1s 230us/step - accuracy: 0.9688 - loss: 0.0436 - val_accuracy: 1.0000 - val_loss: 0.0048
Epoch 9/10
467/467 ━━━━━━━━━━━━━━━━━━━━ 590s 1s/step - accuracy: 0.9766 - loss: 0.0732 - val_accura

Test Loss: 0.0370996929705143
Test Accuracy: 0.984135091304779


In [6]:
# Save the model in the native Keras format
model.save("mymodel.keras")


In [21]:
# for random 50 frames
import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model
import random

# Load the trained model
model = load_model("my_model.h5")

# Function to detect and crop faces from a frame
def crop_faces(frame):
    # Load the Haar cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Crop faces from the frame
    cropped_faces = []
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        cropped_faces.append(face)

    return cropped_faces

# Function to extract frames from a video file
def extract_frames(video_path, num_frames=50):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Choose 50 random frame indices
    random_indices = random.sample(range(frame_count), num_frames)
    
    for idx in random_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            continue

        # Crop faces from the frame
        cropped_faces = crop_faces(frame)

        # Resize and save each cropped face
        for face in cropped_faces:
            if face.shape[0] > 0 and face.shape[1] > 0:
                resized_face = cv2.resize(face, (128, 128))
                frames.append(resized_face)

    cap.release()
    return frames

# Function to predict whether it's a real or deepfake video
def predict_video(video_path):
    # Extract frames from the video
    frames = extract_frames(video_path)

    if len(frames) == 0:
        print("No faces detected in the video.")
        return

    # Convert frames to numpy array and preprocess for prediction
    frames = np.array(frames) / 255.0

    # Predict using the model
    predictions = model.predict(frames)

    # Take the average prediction across frames
    average_prediction = np.mean(predictions)

    # Output the prediction result
    if average_prediction <= 0.5:
        print("Prediction: Deepfake (Confidence: {:.2f}%)".format((1-average_prediction) * 100))
    else:
        print("Prediction: Real (Confidence: {:.2f}%)".format((average_prediction) * 100))

# Example usage
video_path = "C:/Users/rakul/OneDrive/Documents/Python Scripts/face_only/id4_0008.mp4"
predict_video(video_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Prediction: Deepfake (Confidence: 52.51%)


In [36]:
# for random 100 frames
import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model
import random

# Load the trained model
model = load_model("my_model.h5")

# Function to detect and crop faces from a frame
def crop_faces(frame):
    # Load the Haar cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Crop faces from the frame
    cropped_faces = []
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        cropped_faces.append(face)

    return cropped_faces

# Function to extract frames from a video file
def extract_frames(video_path, num_frames=100):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Choose 70 random frame indices
    random_indices = random.sample(range(frame_count), num_frames)
    
    for idx in random_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            continue

        # Crop faces from the frame
        cropped_faces = crop_faces(frame)

        # Resize and save each cropped face
        for face in cropped_faces:
            if face.shape[0] > 0 and face.shape[1] > 0:
                resized_face = cv2.resize(face, (128, 128))
                frames.append(resized_face)

    cap.release()
    return frames

# Function to predict whether it's a real or deepfake video
def predict_video(video_path):
    # Extract frames from the video
    frames = extract_frames(video_path)

    if len(frames) == 0:
        print("No faces detected in the video.")
        return

    # Convert frames to numpy array and preprocess for prediction
    frames = np.array(frames) / 255.0

    # Predict using the model
    predictions = model.predict(frames)

    # Take the average prediction across frames
    average_prediction = np.mean(predictions)

    # Output the prediction result
    if average_prediction <= 0.5:
        print("Prediction: Deepfake (Confidence: {:.2f}%)".format((1-average_prediction) * 100))
    else:
        print("Prediction: Real (Confidence: {:.2f}%)".format((average_prediction) * 100))

# Example usage
video_path = "C:/Users/rakul/OneDrive/Documents/Python Scripts/face_only/id4_0008.mp4"
predict_video(video_path)


2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 6s/step
Prediction: Deepfake (Confidence: 56.01%)


In [37]:
video_path = "Celeb-synthesis/id6_id16_0000.mp4"
predict_video(video_path)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 284ms/step
Prediction: Deepfake (Confidence: 98.99%)


In [38]:
video_path = "YouTube/00239.mp4"
predict_video(video_path)

9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 409ms/step
Prediction: Real (Confidence: 55.46%)


In [39]:
video_path = "Celeb-synthesis/id17_id2_0002.mp4"
predict_video(video_path)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 360ms/step
Prediction: Deepfake (Confidence: 61.43%)


In [40]:
video_path = "C:/Users/rakul/OneDrive/Documents/Python Scripts/face_only/id3_0009.mp4"
predict_video(video_path)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step
Prediction: Real (Confidence: 87.66%)


In [41]:
video_path = "Celeb-real/id3_0007.mp4"
predict_video(video_path)

6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 418ms/step
Prediction: Real (Confidence: 73.56%)


In [42]:
video_path = "Celeb-synthesis/id16_id6_0013.mp4"
predict_video(video_path)

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 318ms/step
Prediction: Deepfake (Confidence: 68.93%)


In [43]:
video_path = "Celeb-real/id2_0002.mp4"
predict_video(video_path)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 374ms/step
Prediction: Real (Confidence: 92.27%)


In [44]:
video_path = "Celeb-synthesis/id10_id12_0004.mp4"
predict_video(video_path)

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 341ms/step
Prediction: Real (Confidence: 61.03%)


In [45]:
video_path = "C:/Users/rakul/Downloads/id0_0000.mp4"
predict_video(video_path)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 331ms/step
Prediction: Real (Confidence: 93.69%)


In [52]:
video_path = "Celeb-synthesis/id7_id12_0000.mp4"
predict_video(video_path)

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 388ms/step
Prediction: Deepfake (Confidence: 77.63%)


In [51]:
video_path = "Celeb-real/id12_0002.mp4"
predict_video(video_path)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 379ms/step
Prediction: Real (Confidence: 95.29%)


In [48]:
video_path = "Celeb-synthesis\id3_id17_0009.mp4"
predict_video(video_path)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 413ms/step
Prediction: Deepfake (Confidence: 91.23%)


In [49]:
video_path = "Celeb-synthesis/id0_id6_0001.mp4"
predict_video(video_path)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 308ms/step
Prediction: Deepfake (Confidence: 89.17%)


In [53]:
import os
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("my_model.h5")

# Define the test directory
test_dir = "C:/Users/rakul/Project/dataset/test"

# Get the list of subdirectories (real and deepfake)
subdirectories = [subdir for subdir in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, subdir))]

# Function to predict on test frames
def predict_test_frames(directory):
    total_videos = 0
    correct_predictions = 0

    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(directory, subdirectory)
        for video_folder in os.listdir(subdirectory_path):
            video_folder_path = os.path.join(subdirectory_path, video_folder)
            frames = os.listdir(video_folder_path)
            num_frames = len(frames)
            if num_frames == 0:
                print(f"No frames found in {video_folder_path}")
                continue
            
            # Randomly select 70 frames for testing
            selected_frames = np.random.choice(frames, size=min(70, num_frames), replace=False)
            frames_data = []
            for frame in selected_frames:
                frame_path = os.path.join(video_folder_path, frame)
                frame_data = cv2.imread(frame_path)
                frames_data.append(frame_data)
            
            # Convert frames to numpy array and preprocess for prediction
            frames_data = np.array(frames_data) / 255.0
            
            # Predict using the model
            predictions = model.predict(frames_data)
            
            # Take the average prediction across frames
            average_prediction = np.mean(predictions)
            
            # Update accuracy based on prediction
            if (average_prediction <= 0.5 and subdirectory == "deepfake") or (average_prediction > 0.5 and subdirectory == "real"):
                correct_predictions += 1
            
            total_videos += 1
    
    # Calculate overall accuracy
    overall_accuracy = (correct_predictions / total_videos) * 100
    print(f"Overall Accuracy: {overall_accuracy:.2f}%")

# Predict on test frames and print overall accuracy
predict_test_frames(test_dir)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 